In [ ]:
import geopandas as gpd
import pandas as pd
import geobr
import matplotlib.pyplot as plt
import numpy as np
from shapely.wkt import loads as load_wkt
from shapely.geometry import Point
import matplotlib.lines as mlines
import re
from PIL import Image

In [ ]:
#Pegar a geometria dos municípios
df_municipios = geobr.read_municipality(code_muni='all', year=2018)
df_municipios

In [ ]:
#Pegando apenas as colunas que correspondem ao código e o shape dos municípios
colunas = ['code_muni', 'geometry']
df_municipios = df_municipios[colunas]
#Retirando valores nulos
df_municipios = df_municipios.dropna()
#Renomear a coluna pra fazer o merge com o csv
df_municipios = df_municipios.rename(columns={'code_muni': 'city_ibge_code'})
#Transforma o code_muni em int para o merge
df_municipios['city_ibge_code'] = df_municipios['city_ibge_code'].astype('int64') 
df_municipios

In [ ]:
#Pegar a geometria dos estados
df_estados = geobr.read_state(code_state='all', year=2018)
df_estados

In [ ]:
#Pegando apenas as colunas que correspondem ao código e o shape dos estados
colunas = ['code_state', 'geometry', 'abbrev_state']
df_estados = df_estados[colunas]
#Retirando valores nulos
df_estados = df_estados.dropna()
#Renomear a coluna pra fazer o merge com o csv
df_estados = df_estados.rename(columns={'code_state': 'city_ibge_code'})
#Transforma o code_state em int para o merge
df_estados['city_ibge_code'] = df_estados['city_ibge_code'].astype('int64') 
df_estados

In [ ]:
#Calcular os centroides dos municípios para centralizar os círculos
centroides = []
for linha in df_municipios.itertuples():
    poligono = load_wkt(str(linha.geometry))
    centroides.append(poligono.centroid)

In [ ]:
#Passar os centroides calculados para um dataframe
df_centroides = pd.DataFrame()
df_centroides['geometry'] = centroides
df_centroides = gpd.GeoDataFrame(df_centroides)
df_centroides.crs = df_municipios.crs

In [ ]:
#Calcular os centroides dos estados para posicionar suas siglas
centroides_estado = []
for linha in df_estados.itertuples():
    poligono = load_wkt(str(linha.geometry))
    centroides_estado.append(poligono.centroid)
    
coordenadas_estado = []
for point in centroides_estado:
    coordenadas_estado.append((point.x, point.y))

#Passar as coordenadas dos centroides para um dataframe
df_estados['centroides'] = coordenadas_estado

In [ ]:
dataset_estados = pd.read_csv("casos_e_mortes_acumulados_estado.csv",index_col=0)
#Retirar a linha com o total do Brasil
dataset_estados = dataset_estados.drop(['TOTAL'])
dataset_estados

In [ ]:
#Adiciona a geometria vinda do geobr
dataset_estados = pd.merge(dataset_estados,df_estados, on='city_ibge_code', how='left')
#Casting para geodataframe
dataset_estados = gpd.GeoDataFrame(dataset_estados)
#Ordenar com base no código do ibge pra combinar com df_estados
dataset_estados = dataset_estados.sort_values(by=['city_ibge_code'])
dataset_estados

In [ ]:
dataset_municipio = pd.read_csv("casos_e_mortes_acumulados_municipio.csv",index_col=0)
dataset_municipio

In [ ]:
#Adiciona a geometria vinda do geobr
dataset_municipio = pd.merge(dataset_municipio,df_municipios, on='city_ibge_code', how='left')
#Casting para geodataframe
dataset_municipio = gpd.GeoDataFrame(dataset_municipio)
dataset_municipio

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de casos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))

dados = dataset_municipio[['geometry', 'last_available_confirmed_week_31']]

markersize = []
for linha in dados.itertuples():
    markersize.append(linha.last_available_confirmed_week_31/10)
    
df_municipios.plot(ax=ax, facecolor='#cccccc')
df_estados.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)

#Pegar as coordenadas dos centroides
xs=[point.x for point in df_centroides['geometry']]
ys=[point.y for point in df_centroides['geometry']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=6)
for i in range(len(labels)):
    labels[i]= labels[i][:-3] + labels[i][-4:]

In [ ]:
#Definição da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2, handlelength=7, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plotar a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    
    #Pegar apenas as colunas necessárias (geometry e quantidade de casos)
    dados = dataset_municipio[['geometry', 'last_available_confirmed_week_{}'.format(semana)]]
    dados = dados.rename(columns={'last_available_confirmed_week_{}'.format(semana): 'casos'})

    #Definir o tamanho de cada círculo
    markersize = []
    for linha in dados.itertuples():
        markersize.append(linha.casos/10)

    #Plotar as linhas dos limites dos municipios
    df_municipios.plot(ax=ax, facecolor='#cccccc')
    df_estados.geometry.boundary.plot(color=None,edgecolor='#444444',linewidth = 0.2,ax=ax)
    
    #Plotar os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escrever a sigla de cada estado
    for linha in df_estados.itertuples():
        plt.annotate(s=linha.abbrev_state, xy=linha.centroides, horizontalalignment='center', verticalalignment='center', fontsize=20, color='#444444')

    #Executar a definição prévia da legenda
    exec(legend)
    #Definir o título
    plt.title("Casos confirmados por município - Semana {}".format(semana), fontsize=30)
    #Salvar o arquivo da imagem
    plt.savefig("mapa_casos_municipios_{}.png".format(semana))
    im = Image.open("mapa_casos_municipios_{}.png".format(semana))
    im = im.crop((150,120,1290,1270))
    im.save("mapa_casos_municipios_{}.png".format(semana))

In [ ]:
images = []
for semana in range(9,32):
    images.append(Image.open("mapa_casos_municipios_{}.png".format(semana)))
        
images[0].save('mapa_casos_municipios.gif', save_all=True, append_images=images, duration=180, loop=0)

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de casos por estado, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))

dados = dataset_estados[['geometry', 'last_available_confirmed_week_31']]

markersize = []
for linha in dados.itertuples():
    markersize.append(linha.last_available_confirmed_week_31/25)
    
df_municipios.plot(ax=ax, facecolor='#cccccc')
df_estados.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)

#Pegar as coordenadas dos centroides
xs=[point[0] for point in df_estados['centroides']]
ys=[point[1] for point in df_estados['centroides']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*25)

In [ ]:
#Definição da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2, handlelength=7, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plotar a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    
    #Pegar apenas as colunas necessárias (geometry e quantidade de casos)
    dados = dataset_estados[['geometry', 'last_available_confirmed_week_{}'.format(semana)]]
    dados = dados.rename(columns={'last_available_confirmed_week_{}'.format(semana): 'casos'})

    #Definir o tamanho de cada círculo
    markersize = []
    for linha in dados.itertuples():
        markersize.append(linha.casos/25)

    #Plotar as linhas dos limites dos municipios
    df_municipios.plot(ax=ax, facecolor='#cccccc')
    df_estados.geometry.boundary.plot(color=None,edgecolor='#444444',linewidth = 0.2,ax=ax)
    
    #Plotar os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escrever a sigla de cada estado
    for linha in df_estados.itertuples():
        plt.annotate(s=linha.abbrev_state, xy=linha.centroides, horizontalalignment='center', verticalalignment='center', fontsize=20, color='#444444')

    #Executar a definição prévia da legenda
    exec(legend)
    #Definir o título
    plt.title("Casos confirmados por estado - Semana {}".format(semana), fontsize=30)
    #Salvar o arquivo da imagem
    plt.savefig("mapa_casos_estados_{}.png".format(semana))
    im = Image.open("mapa_casos_estados_{}.png".format(semana))
    im = im.crop((150,120,1290,1270))
    im.save("mapa_casos_estados_{}.png".format(semana))

In [ ]:
images = []
for semana in range(9,32):
    images.append(Image.open("mapa_casos_estados_{}.png".format(semana)))
        
images[0].save('mapa_casos_estados.gif', save_all=True, append_images=images, duration=180, loop=0)

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de mortes por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))

dados = dataset_municipio[['geometry', 'last_available_deaths_week_31']]

markersize = []
for linha in dados.itertuples():
    markersize.append(linha.last_available_deaths_week_31)
    
df_municipios.plot(ax=ax, facecolor='#cccccc')
df_estados.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)

#Pegar as coordenadas dos centroides
xs=[point.x for point in df_centroides['geometry']]
ys=[point.y for point in df_centroides['geometry']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=6)

In [ ]:
#Definição da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2, handlelength=7, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plotar a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    
    #Pegar apenas as colunas necessárias (geometry e quantidade de casos)
    dados = dataset_municipio[['geometry', 'last_available_deaths_week_{}'.format(semana)]]
    dados = dados.rename(columns={'last_available_deaths_week_{}'.format(semana): 'mortes'})

    #Definir o tamanho de cada círculo
    markersize = []
    for linha in dados.itertuples():
        markersize.append(linha.mortes)

    #Plotar as linhas dos limites dos municipios
    df_municipios.plot(ax=ax, facecolor='#cccccc')
    df_estados.geometry.boundary.plot(color=None,edgecolor='#444444',linewidth = 0.2,ax=ax)
    
    #Plotar os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escrever a sigla de cada estado
    for linha in df_estados.itertuples():
        plt.annotate(s=linha.abbrev_state, xy=linha.centroides, horizontalalignment='center', verticalalignment='center', fontsize=20, color='#444444')

    #Executar a definição prévia da legenda
    exec(legend)
    #Definir o título
    plt.title("Mortes confirmadas por município - Semana {}".format(semana), fontsize=30)
    #Salvar o arquivo da imagem
    plt.savefig("mapa_mortes_municipios_{}.png".format(semana))
    im = Image.open("mapa_mortes_municipios_{}.png".format(semana))
    im = im.crop((150,120,1290,1270))
    im.save("mapa_mortes_municipios_{}.png".format(semana))

In [ ]:
images = []
for semana in range(9,32):
    images.append(Image.open("mapa_mortes_municipios_{}.png".format(semana)))
        
images[0].save('mapa_mortes_municipios.gif', save_all=True, append_images=images, duration=180, loop=0)

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de mortes por estado, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))

dados = dataset_estados[['geometry', 'last_available_deaths_week_31']]

markersize = []
for linha in dados.itertuples():
    markersize.append(linha.last_available_deaths_week_31)
    
df_municipios.plot(ax=ax, facecolor='#cccccc')
df_estados.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)

#Pegar as coordenadas dos centroides
xs=[point[0] for point in df_estados['centroides']]
ys=[point[1] for point in df_estados['centroides']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=5)

In [ ]:
#Definição da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2, handlelength=7, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plotar a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    
    #Pegar apenas as colunas necessárias (geometry e quantidade de casos)
    dados = dataset_estados[['geometry', 'last_available_deaths_week_{}'.format(semana)]]
    dados = dados.rename(columns={'last_available_deaths_week_{}'.format(semana): 'mortes'})

    #Definir o tamanho de cada círculo
    markersize = []
    for linha in dados.itertuples():
        markersize.append(linha.mortes)

    #Plotar as linhas dos limites dos municipios
    df_municipios.plot(ax=ax, facecolor='#cccccc')
    df_estados.geometry.boundary.plot(color=None,edgecolor='#444444',linewidth = 0.2,ax=ax)
    
    #Plotar os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escrever a sigla de cada estado
    for linha in df_estados.itertuples():
        plt.annotate(s=linha.abbrev_state, xy=linha.centroides, horizontalalignment='center', verticalalignment='center', fontsize=20, color='#444444')

    #Executar a definição prévia da legenda
    exec(legend)
    #Definir o título
    plt.title("Mortes confirmadas por estado - Semana {}".format(semana), fontsize=30)
    #Salvar o arquivo da imagem
    plt.savefig("mapa_mortes_estados_{}.png".format(semana))
    im = Image.open("mapa_mortes_estados_{}.png".format(semana))
    im = im.crop((150,120,1290,1270))
    im.save("mapa_mortes_estados_{}.png".format(semana))

In [ ]:
images = []
for semana in range(9,32):
    images.append(Image.open("mapa_mortes_estados_{}.png".format(semana)))
        
images[0].save('mapa_mortes_estados.gif', save_all=True, append_images=images, duration=180, loop=0)